### Welcome to the ProtoSyn.jl examples

# 4 - Energy Calculation

When comparing two or more molecular structures, it's useful to have a way to measure the fitness (or how "real" a structure is). This is often achieved by applying an energy funcion: a set of energy component calculations summed to return a fitness gauge. In ProtoSyn, this is easily achieved by applying the default `EnergyFunction` or creating a new or custom function. For this example, we will load the 2A3D proteic structure.

In [33]:
using ProtoSyn

In [11]:
pose = ProtoSyn.Peptides.load("data/2a3d.pdb")

┌ Info: Flag `bonds_by_distance` is set to False. Make sure the loaded data/2a3d.pdb file has connect records.
└ @ ProtoSyn.Peptides /home/jpereira/project_c/ProtoSyn.jl/src/Peptides/Methods/io.jl:30


Pose{Topology}(Topology{/2a3d:14695}, State{Float64}:
 Size: 1140
 i2c: false | c2i: false
 Energy: Dict(:Total => Inf)
)

In [12]:
energy_function = ProtoSyn.Common.default_energy_function()

🗲  Energy Function (4 components):
+----------------------------------------------------------------------+
| Index | Component name                                | Weight (α)   |
+----------------------------------------------------------------------+
| 1     | TorchANI_ML_Model                             |      1.000   |
| 2     | Caterpillar_Solvation                         |      0.010   |
| 3     | Bond_Distance_Restraint                       |      1.000   |
| 4     | Cα-Cα_Clash_Restraint                         |    100.000   |
+----------------------------------------------------------------------+


In [13]:
energy_function(pose)

-6.597528783285798

Since our energy function had multiple energy components, we can analyze each individual contribution in the `pose.state.e` dictionary.

In [14]:
pose.state.e

Dict{Symbol, Float64} with 5 entries:
  :Total                          => -6.59753
  :TorchANI_ML_Model              => -9.84547
  :Bond_Distance_Restraint        => 0.0
  :Caterpillar_Solvation          => 3.24794
  Symbol("Cα-Cα_Clash_Restraint") => 0.0

By setting the `update_forces` flag to true, when calling the energy function, we can also calculate the forces felt on each atom of the Pose.

In [15]:
energy_function(pose, true)

-6.597528783285798

In [16]:
pose.state.f

3×1140 Matrix{Float64}:
 -0.0334201   0.0380794   -0.0653166  …  -0.0110561   0.0583287   -0.0354509
 -0.012151    0.00233961  -0.0503394      0.023544   -0.0214769   -0.0355987
 -0.00557967  0.016839     0.0254311      0.0118799  -0.00884128   0.0172025

The energy function is completly modifiable, the user can add or remove components, as well as change the weight bias given to each one, as exemplified bellow. 

In [23]:
energy_function["Caterpillar_Solvation"].α = 0.03;
energy_function

🗲  Energy Function (4 components):
+----------------------------------------------------------------------+
| Index | Component name                                | Weight (α)   |
+----------------------------------------------------------------------+
| 1     | TorchANI_ML_Model                             |      1.000   |
| 2     | Caterpillar_Solvation                         |      0.030   |
| 3     | Bond_Distance_Restraint                       |      1.000   |
| 4     | Cα-Cα_Clash_Restraint                         |    100.000   |
+----------------------------------------------------------------------+


In [24]:
energy_function(pose)

-0.10164740027243191

As we can see, the resulting energy is now different. Besides altering the `α` weight bias, some `EnergyFunctionComponent` instances have a set of settings, specific for each one, that can be fine tuned. for example, we can set the bond distance restraint to a lower distance.

In [31]:
energy_function["Bond_Distance_Restraint"].settings[:x0] = 0.2
energy_function["Bond_Distance_Restraint"]

          Name : Bond_Distance_Restraint
    Weight (α) : 1.0
 Update forces : true
       Setings :
            :x0 => 0.2


In [32]:
energy_function(pose)

365.8297953062032

Which, of course, results in an absurd increase in energy of the system.

## Conclusion

In this brief example we took a look at how to gauge the fitness of a structure, using the default Energy Function instance of ProtoSyn, as well as how to modify and fine tune each of the individual energy function component instances that compose the energy function.